#**PANUGANTI SAI LIKHITH**

sailikithpaanuganti@gmail.com

https://www.linkedin.com/in/sailikhith/

# **Problem statement**

Create a classification model to predict the sentiment either (Positive or Negative) based on Covid Tweets

# **Dataset Required**

https://github.com/Sai-Likhith/Sentiment-Analysis-of-COVID-19-Tweets-A-Comparative-Analysis-of-Classification-Models/blob/main/Corona_NLP.csv

#**Getting Started**

In [8]:
#Importing Required

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.stem.porter import *
import re
import seaborn as sns
palette=sns.color_palette('magma')
sns.set(palette=palette)

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import sklearn.model_selection, sklearn.svm, sklearn.neighbors, sklearn.metrics

import matplotlib.pyplot as plt

# **Read the dataset with encoding parameter set to ‘latin1’**

In [9]:
df = pd.read_csv('/content/Corona_NLP.csv', encoding='latin1')

In [10]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [11]:
df.shape

(28667, 6)

In [12]:
df.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

# **Checking for any null values and removing (if any)**

In [13]:
pd.isnull(df).sum()

UserName            0
ScreenName          0
Location         6033
TweetAt             0
OriginalTweet       0
Sentiment           1
dtype: int64

In [14]:
df = df.dropna( axis=0,inplace=False)    #Removes the Null data if any

In [15]:
pd.isnull(df).sum()

UserName         0
ScreenName       0
Location         0
TweetAt          0
OriginalTweet    0
Sentiment        0
dtype: int64

# **Preprocess the Covid tweets based on the following parameter:**
Tokenizing words, Convert words to lower case, Removing Punctuations, Removing Stop words, Stemming or lemmatizing the words

***Removing special characters along with @mentions ,http:// and puntuations***

---



In [16]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i,'',input_txt)
    return input_txt

In [17]:
df['OriginalTweet'] = np.vectorize(remove_pattern)(df['OriginalTweet'], '@[\w]*')
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df['OriginalTweet'] = df['OriginalTweet'].str.replace('[^a-zA-Z#]+',' ')

<ipython-input-17-55a11a509d84>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['OriginalTweet'] = df['OriginalTweet'].str.replace('[^a-zA-Z#]+',' ')


***Removing Stop Words***

---



In [18]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
df['OriginalTweet'].apply(lambda x: [item for item in x if item not in stop])

0                                                      [ ]
1        [v, c, e,  , T, l, k,  ,  , u, r,  , n, e, g, ...
2        [C, r, n, v, r, u,  , A, u, r, l,  , W, l, w, ...
5        [A,  , n, e, w,  , f,  , h, e,  , r, e, g, n, ...
6        [C, h, e, r,  ,  , g, r, c, e, r,  , r, e,  , ...
                               ...                        
28661    [e,  ,  , w, r, k,  , w, h, e, n,  , p, e, p, ...
28662    [H, e, l, h,  , n, u, r, n, c, e,  , p, r, e, ...
28663    [T, h, e,  ,  , g, r, n, p, r, e, n,  , n,  , ...
28664    [ , n,  , h,  ,  , j, u,  , f, r,  , C, v,  , ...
28665    [I, f,  , u,  , c, n,  , f, n,  , T, P,  , #, ...
Name: OriginalTweet, Length: 22633, dtype: object

***Converting OriginalTweet words to lower case***

---



In [20]:
# Convert words to lower case

df['OriginalTweet'] = df['OriginalTweet'].str.lower()

***Tokenizing OriginalTweet words***

---



In [21]:
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [22]:
df['OriginalTweet'] = df['OriginalTweet'].apply(word_tokenize)

In [23]:
df['OriginalTweet']

0                                                       []
1        [advice, talk, to, your, neighbours, family, t...
2        [coronavirus, australia, woolworths, to, give,...
5        [as, news, of, the, region, s, first, confirme...
6        [cashier, at, grocery, store, was, sharing, hi...
                               ...                        
28661    [me, at, work, when, people, ask, if, we, have...
28662    [health, insurance, premiums, could, rise, up,...
28663    [the, says, grandparent, and, family, emergenc...
28664    [and, that, s, just, from, covid, low, oil, pr...
28665    [if, u, cant, find, tp, #, toiletpaper, at, th...
Name: OriginalTweet, Length: 22633, dtype: object

***Stemming the OriginalTweet words***

---



In [24]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x: [stemmer.stem(i) for i in x])
df['OriginalTweet'] = df['OriginalTweet'].str.join(' ')

In [25]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,,Neutral
1,3800,48752,UK,16-03-2020,advic talk to your neighbour famili to exchang...,Positive
2,3801,48753,Vagabonds,16-03-2020,coronaviru australia woolworth to give elderli...,Positive
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,as news of the region s first confirm covid ca...,Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,cashier at groceri store wa share hi insight o...,Positive


# **Convert 'Extremely Positive', 'Extremely Negative' Sentiments to 'Positive', 'Negative' sentiments respectively**

In [26]:
Sentiment_encoding = {
    'Extremely Negative': 0,
    'Negative': 0,
    'Neutral': 1,
    'Positive': 2,
    'Extremely Positive': 2
}
df['Sentiment']=df['Sentiment'].replace(Sentiment_encoding)

In [27]:
y = df['Sentiment']

In [28]:
y.unique()

array([1, 2, 0])

In [29]:
df.head(10)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,,1
1,3800,48752,UK,16-03-2020,advic talk to your neighbour famili to exchang...,2
2,3801,48753,Vagabonds,16-03-2020,coronaviru australia woolworth to give elderli...,2
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,as news of the region s first confirm covid ca...,2
6,3805,48757,"35.926541,-78.753267",16-03-2020,cashier at groceri store wa share hi insight o...,2
7,3806,48758,Austria,16-03-2020,wa at the supermarket today didn t buy toilet ...,1
8,3807,48759,"Atlanta, GA USA",16-03-2020,due to covid our retail store and classroom in...,2
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,for corona prevent we should stop to buy thing...,0
10,3809,48761,"Makati, Manila",16-03-2020,all month there hasn t been crowd in the super...,1
11,3810,48762,"Pitt Meadows, BC, Canada",16-03-2020,due to the covid situat we have increas demand...,2


# **Splitting data into training and test data.**

In [30]:
data = df[['OriginalTweet','Sentiment']]
data.shape

(22633, 2)

In [31]:
train,test = train_test_split(data,test_size = 0.2,random_state=0,stratify = data.Sentiment.values)

print("Train shape : ", train.shape)
print("Test shape : ", test.shape)

Train shape :  (18106, 2)
Test shape :  (4527, 2)


# **Transform the words into vectors using Count Vectorizer**


In [32]:
stop = list(stopwords.words('english'))
vectorizer = CountVectorizer(decode_error = 'replace',stop_words = stop)

X_train = vectorizer.fit_transform(train.OriginalTweet.values)
X_test = vectorizer.transform(test.OriginalTweet.values)

y_train = train.Sentiment.values
y_test = test.Sentiment.values

In [33]:
print("X_train.shape : ", X_train.shape)
print("X_test.shape : ", X_test.shape)
print("y_train.shape : ", y_train.shape)
print("y_test.shape : ", y_test.shape)

X_train.shape :  (18106, 19135)
X_test.shape :  (4527, 19135)
y_train.shape :  (18106,)
y_test.shape :  (4527,)


# **Apply the following models on the training dataset and generate the predicted value for the test dataset**

a) Multinomial Naïve Bayes Classification

b) SVM Classification

c) KNN Classification


**a) Multinomial Naïve Bayes Classification**


---


In [34]:
naiveByes_clf = MultinomialNB()
naiveByes_clf.fit(X_train,y_train)

NB_prediction = naiveByes_clf.predict(X_test)
NB_accuracy = accuracy_score(y_test,NB_prediction)
print("training accuracy Score    : ",naiveByes_clf.score(X_train,y_train))
print("testing accuracy Score : ",NB_accuracy )
print(classification_report(NB_prediction,y_test))

training accuracy Score    :  0.7951507787473766
testing accuracy Score :  0.6556218246079081
              precision    recall  f1-score   support

           0       0.71      0.67      0.69      1850
           1       0.19      0.62      0.29       253
           2       0.80      0.65      0.72      2424

    accuracy                           0.66      4527
   macro avg       0.57      0.65      0.57      4527
weighted avg       0.73      0.66      0.68      4527



In [35]:
confusion_matrix_nb = confusion_matrix(y_test,NB_prediction)

**b) SVM Classification**

---



In [36]:
svm = svm.LinearSVC(multi_class='ovr', max_iter = 10000)
svm.fit(X_train, y_train)

svm_prediction = svm.predict(X_test)
svm_accuracy = accuracy_score(y_test,svm_prediction)
print("Training accuracy Score    : ",svm.score(X_train,y_train))
print("Testing accuracy Score : ",svm_accuracy )
print(classification_report(svm_prediction,y_test))

Training accuracy Score    :  0.9830995250193306
Testing accuracy Score :  0.7506074663132317
              precision    recall  f1-score   support

           0       0.76      0.78      0.77      1686
           1       0.61      0.60      0.61       849
           2       0.80      0.79      0.80      1992

    accuracy                           0.75      4527
   macro avg       0.72      0.72      0.72      4527
weighted avg       0.75      0.75      0.75      4527



In [37]:
confusion_matrix_svm = confusion_matrix(y_test,svm_prediction)

**c) KNN Classification**

---



In [38]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

knn_prediction = knn.predict(X_test)
knn_accuracy = accuracy_score(y_test,knn_prediction)
print("Training accuracy Score    : ",knn.score(X_train,y_train))
print("Testing accuracy Score : ",knn_accuracy )
print(classification_report(knn_prediction,y_test))

Training accuracy Score    :  0.5803601016237712
Testing accuracy Score :  0.340622929092114
              precision    recall  f1-score   support

           0       0.27      0.62      0.38       770
           1       0.87      0.22      0.35      3259
           2       0.18      0.70      0.28       498

    accuracy                           0.34      4527
   macro avg       0.44      0.51      0.34      4527
weighted avg       0.69      0.34      0.35      4527



In [39]:
confusion_matrix_knn = confusion_matrix(y_test,knn_prediction)

# **Compute Confusion matrix and classification report for each of these models**

In [40]:
# Confusion matrix for Multinomial Naïve Bayes

print("Confusion matrix for Multinomial Naïve Bayes Classification is : ")
print(confusion_matrix_nb)
print('\n')

#Classification Report for Multinomial Naïve Bayes

target_names = np.unique(y)
target_names = ['0', '1', '2']
NB_prediction.shape
print("Classification Report for Multinomial Naïve Bayes Classification is : \n")
print(classification_report(y_test.ravel() , NB_prediction.ravel(), target_names=target_names))

Confusion matrix for Multinomial Naïve Bayes Classification is : 
[[1234   46  453]
 [ 275  158  395]
 [ 341   49 1576]]


Classification Report for Multinomial Naïve Bayes Classification is : 

              precision    recall  f1-score   support

           0       0.67      0.71      0.69      1733
           1       0.62      0.19      0.29       828
           2       0.65      0.80      0.72      1966

    accuracy                           0.66      4527
   macro avg       0.65      0.57      0.57      4527
weighted avg       0.65      0.66      0.63      4527



In [41]:
# Confusion matrix for SVM

print("Confusion matrix for SVM  is : ")
print(confusion_matrix_svm)
print('\n')

#Classification Report for SVM

target_names = np.unique(y)
target_names = ['0', '1', '2']
NB_prediction.shape
print("Classification Report for SVM Classification is : \n")
print(classification_report(y_test.ravel() , svm_prediction.ravel(), target_names=target_names))

Confusion matrix for SVM  is : 
[[1311  172  250]
 [ 155  509  164]
 [ 220  168 1578]]


Classification Report for SVM Classification is : 

              precision    recall  f1-score   support

           0       0.78      0.76      0.77      1733
           1       0.60      0.61      0.61       828
           2       0.79      0.80      0.80      1966

    accuracy                           0.75      4527
   macro avg       0.72      0.72      0.72      4527
weighted avg       0.75      0.75      0.75      4527



In [42]:
# Confusion matrix for KNN

print("Confusion matrix for KNN is : ")
print(confusion_matrix_knn)
print('\n')

#Classification Report for KNN

target_names = np.unique(y)
target_names = ['0', '1', '2']
NB_prediction.shape
print("Classification Report for KNN Classification is : \n")
print(classification_report(y_test.ravel() , knn_prediction.ravel(), target_names=target_names))

Confusion matrix for KNN is : 
[[ 475 1155  103]
 [  60  720   48]
 [ 235 1384  347]]


Classification Report for KNN Classification is : 

              precision    recall  f1-score   support

           0       0.62      0.27      0.38      1733
           1       0.22      0.87      0.35       828
           2       0.70      0.18      0.28      1966

    accuracy                           0.34      4527
   macro avg       0.51      0.44      0.34      4527
weighted avg       0.58      0.34      0.33      4527



# **Report the model with the best accuracy.**

In [60]:
models = pd.DataFrame({
    'Model': ['Naive Bayes', 'Support Vector Machines',' KNN Classification'],
    'Accuracy': [NB_accuracy*100, svm_accuracy*100, knn_accuracy*100]})

models.sort_values(by='Accuracy', ascending=False).reset_index(drop=True)

,Model,Accuracy
0,Support Vector Machines,75.060747
1,Naive Bayes,65.562182
2,KNN Classification,34.062293


**Support Vector Machines (SVM) got the highes accuracy, whereas Naives Bayes is in the Second place of best accuracy.**

The least among all the three models is KNN which is not preferable for this dataset.